1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다
   - 토큰 수 초과로 답변을 생성하지 못할 수 있음
   - 문서가 길면 (input이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달


In [1]:
# DOCX 파일을 로드하기 위한 로더 생성
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

# tax.docx 파일을 읽을 로더 객체 생성
loader = Docx2txtLoader("./tax.docx")

# 문서 로드 (Document 객체 리스트 반환)
document_list = loader.load_and_split(text_splitter=text_splitter)

In [2]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()  # .env 파일에서 환경 변수 로드

embedding = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

In [7]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pinecone_api_key = os.getenv("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)
index_name = "tax-index"

database = PineconeVectorStore.from_documents(
    document_list,
    embedding,
    index_name=index_name,
)

In [ ]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"

# 유사도 검색 테스트
retrieved_docs = database.similarity_search(query, k=3)

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [11]:
prompt = f"""
[Identity]

- 당신은 최고의 한국 소득세 전문가입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.

[Context]
{retrieved_docs}

Question: {query}
"""

In [12]:
ai_message = llm.invoke(prompt)

ai_message

AIMessage(content='연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 한국의 소득세 계산 방법을 따라야 합니다. 2023년 기준의 소득세율을 적용하여 대략적인 세액을 계산해 보겠습니다.\n\n한국의 소득세는 누진세율 구조로 되어 있으며, 2023년 기준으로 다음과 같은 세율이 적용됩니다:\n\n1. 1,200만원 이하: 6%\n2. 1,200만원 초과 ~ 4,600만원: 15%\n3. 4,600만원 초과 ~ 8,800만원: 24%\n\n연봉 5천만원인 경우를 계산하면:\n\n- 1,200만원까지: 1,200만원 × 6% = 72만원\n- 1,200만원 초과 4,600만원까지 (3,400만원):\n  - 3,400만원 × 15% = 510만원\n- 4,600만원 초과 5,000만원까지 (400만원):\n  - 400만원 × 24% = 96만원\n\n이제 세액을 합산합니다:\n\n- 72만원 + 510만원 + 96만원 = 678만원\n\n따라서, 연봉 5천만원인 직장인의 소득세는 대략 **678만원**입니다.\n\n다만, 여기에는 기본공제, 인적공제 및 세액공제 등이 반영되지 않은 단순 계산입니다. 최종 세액은 개인의 상황에 따라 달라질 수 있습니다. 추가적인 공제가 필요하시면, 개별적인 정보를 제공해 주시면 더 구체적인 계산을 도와드리겠습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 360, 'prompt_tokens': 3047, 'total_tokens': 3407, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_token